# Smoke Plume

This example simulates a rising plume of hot smoke. It demonstrates how to advect and couple other quantities with the fluid.

- https://tum-pbs.github.io/PhiFlow/examples/grids/Smoke_Plume.html

In [ ]:
from phi.jax.flow import *
from tqdm.notebook import trange

In [ ]:
domain = Box(x=100, y=100)
inflow = Sphere(x=50, y=9.5, radius=5)
inflow_rate = 0.2

In [ ]:
@jit_compile
def step(v, s, p, dt):
    s = advect.mac_cormack(s, v, dt) + inflow_rate * resample(inflow, to=s, soft=True)
    buoyancy = resample(s * (0, 0.1), to=v)
    v = advect.semi_lagrangian(v, v, dt) + buoyancy * dt
    v, p = fluid.make_incompressible(v, (), Solve('CG', 1e-3, x0=p))
    return v, s, p

v0 = StaggeredGrid(0, 0, domain, x=64, y=64)
smoke0 = CenteredGrid(0, ZERO_GRADIENT, domain, x=200, y=200)

In [ ]:
v_trj, s_trj, p_trj = iterate(step, batch(time=300), v0, smoke0, None, dt=.5, range=trange, substeps=3)

  0%|          | 0/300 [00:00<?, ?it/s]

In [ ]:
plot(s_trj, animate='time', frame_time=80)

<Figure size 640x480 with 0 Axes>